In [1]:
!pip install pandas openpyxl xlrd


Defaulting to user installation because normal site-packages is not writeable


In [32]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Admin\Downloads\JOB WORK INVOICE SHM023 DT19062025 SHIM JOB WORK-LGD.xlsx', sheet_name='PO Copy.',skiprows=2)


selected_columns = ['Vendor Style#', 'SIZE', 'Ord Qty', 
                    'KT/Col', 'NY PO#','Spl. Remark','Min / Max','Stamp Instr','Cust','NY Style #','LGD QUALITY','Mined/LGD','Indigo Order#']

# Explicit copy
df_selected = df[selected_columns].copy()

# Rename columns
df_selected.rename(columns={
    'Vendor Style#': 'StyleCode',
    'SIZE': 'ItemSize',
    'Ord Qty': 'OrderQty',
    'NY PO#': 'ItemPoNo',
    'Spl. Remark': 'CustomerProductionInstruction',
    'Stamp Instr' : 'StampInstruction',
    'Cust' : 'OrderGroup',
    'NY Style #' : 'SKUNo'
}, inplace=True)

df_selected.insert(
    df_selected.columns.get_loc('StyleCode'),
    'SrNo',
    range(1, len(df_selected) + 1)  # incremental numbers starting from 1
)

# Function to map KT/Col to Metal code
def metal_code(kt_col):
    if pd.isna(kt_col):
        return ''
    
    kt_col = str(kt_col).upper().strip()
    
    # Platinum
    if '950PT' in kt_col:
        return 'PT950'
    
    # Karat gold
    if 'KT' in kt_col:
        if '/' in kt_col:
            karat, color = kt_col.split('/')
            karat = karat.replace('KT','')
            if color == 'W':
                return f'G{karat}W'
            elif color == 'Y':
                return f'G{karat}Y'
            elif color == 'P':
                return f'G{karat}P'
            else:
                return f'G{karat}X'  # unknown color
        else:
            karat = kt_col.replace('KT','')
            return f'G{karat}X'
    
    return 'UNKNOWN'

# Insert Metal column after OrderQty
df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'Metal',
    df_selected['KT/Col'].apply(metal_code)
)

# Function to extract tone from KT/Col
def extract_tone(kt_col):
    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip()
    if '/' in kt_col:
        return kt_col.split('/')[-1]  # take last part after "/"
    return ''  # if no slash, return blank

# Insert Tone column after Metal
df_selected.insert(
    df_selected.columns.get_loc('Metal') + 1,
    'Tone',
    df_selected['KT/Col'].apply(extract_tone)
)

# Create DesignProductionInstruction by combining CustomerProductionInstruction and Min / Max
def design_instruction(row):
    cpi = '' if pd.isna(row['CustomerProductionInstruction']) else str(row['CustomerProductionInstruction']).strip()
    minmax = '' if pd.isna(row['Min / Max']) else str(row['Min / Max']).strip()
    
    if cpi and minmax:
        return f"{cpi}, {minmax}"
    elif cpi:
        return cpi
    elif minmax:
        return minmax
    else:
        return ''

# Insert DesignProductionInstruction column after Tone
df_selected.insert(
    df_selected.columns.get_loc('Tone') + 1,
    'DesignProductionInstruction',
    df_selected.apply(design_instruction, axis=1)
)

# Function to create SpecialRemarks
def create_special_remarks(row):
    parts = []
    # Add each part if it exists
    if not pd.isna(row['OrderGroup']):
        parts.append(str(row['OrderGroup']).strip())
    if not pd.isna(row['StyleCode']):
        parts.append(str(row['StyleCode']).strip())
    if not pd.isna(row['KT/Col']):
        parts.append(str(row['KT/Col']).strip())
    if not pd.isna(row['LGD QUALITY']):
        parts.append(str(row['LGD QUALITY']).strip())
    if not pd.isna(row['Mined/LGD']):
        parts.append(str(row['Mined/LGD']).strip())
    if not pd.isna(row['Indigo Order#']):
        parts.append(str(row['Indigo Order#']).strip())
    
    return ', '.join(parts)

# Insert SpecialRemarks column after CustomerProductionInstruction
df_selected.insert(
    df_selected.columns.get_loc('CustomerProductionInstruction') + 1,
    'SpecialRemarks',
    df_selected.apply(create_special_remarks, axis=1)
)

df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'OrderItemPcs',
    ''  # blank by default
)

# Move ItemPoNo column after Tone
item_po_no = df_selected.pop('ItemPoNo')
df_selected.insert(df_selected.columns.get_loc('Tone') + 1, 'ItemPoNo', item_po_no)

# List of new columns to add after ItemPoNo
new_cols = ['ItemRefNo', 'StockType', 'MakeType']

# Insert each new column as blank after ItemPoNo
pos = df_selected.columns.get_loc('ItemPoNo') + 1
for col in new_cols:
    df_selected.insert(pos, col, '')
    pos += 1
    
# Move DesignProductionInstruction after SpecialRemarks
design_col = df_selected.pop('DesignProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('SpecialRemarks') + 1, 'DesignProductionInstruction', design_col)

# List of additional new columns to add after SKUNo
additional_cols = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]

# Insert each new column as blank after SKUNo
pos = df_selected.columns.get_loc('SKUNo') + 1
for col in additional_cols:
    df_selected.insert(pos, col, '')
    pos += 1

df_selected.drop(columns=['Mined/LGD','Indigo Order#','KT/Col','Min / Max','LGD QUALITY'], inplace=True) 
#df_selected.to_csv(r'C:\Users\Admin\Desktop\SHM023 DT19062025_PO_Cleaned.csv', index=False)
df_selected.head()

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,OrderGroup,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,VR4178SEB1,7.00,7,,G14W,W,208538,,,...,REED,B1155BPC1.00RJ,,,,,,,,
1,2,XR3843L,7.00,1,,G10 X,W,408815,,,...,HZ,LGF4060-10W-18A,,,,,,,,
2,3,XR5357ME,10.50,1,,G14 X,Y,408817,,,...,RH,B1342AE2.0RH,,,,,,,,
3,4,WR4335JG,10.00,1,,G14 X,YW,409510-BN,,,...,HZ,LGG4010-14YW-96A,,,,,,,,
4,5,SVBRC24367XX,6.75,1,,UNKNOWN,W,208704,,,...,IDI,-,,,,,,,,


In [35]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Admin\Downloads\Po# Mounting-1.xlsx', sheet_name='PO Copy.',skiprows=1)


selected_columns = ['Vendor Style#', 'SIZE', 'Ord Qty', 
                    'KT/Col', 'NY PO#','Spl. Remark','Min / Max','Stamp Instr','Cust','NY Style #','LGD QUALITY','Mined/LGD','Indigo Order#']

# Explicit copy
df_selected = df[selected_columns].copy()

# Rename columns
df_selected.rename(columns={
    'Vendor Style#': 'StyleCode',
    'SIZE': 'ItemSize',
    'Ord Qty': 'OrderQty',
    'NY PO#': 'ItemPoNo',
    'Spl. Remark': 'CustomerProductionInstruction',
    'Stamp Instr' : 'StampInstruction',
    'Cust' : 'OrderGroup',
    'NY Style #' : 'SKUNo'
}, inplace=True)

df_selected.insert(
    df_selected.columns.get_loc('StyleCode'),
    'SrNo',
    range(1, len(df_selected) + 1)  # incremental numbers starting from 1
)

# Function to map KT/Col to Metal code
def metal_code(kt_col):
    if pd.isna(kt_col):
        return ''
    
    kt_col = str(kt_col).upper().strip().replace(' ', '')  # remove spaces
    
    # Platinum
    if '950PT' in kt_col or 'PT950' in kt_col:
        return 'PC95'
    
    # Karat gold
    if 'KT' in kt_col:
        if '/' in kt_col:
            karat, color = kt_col.split('/')
            karat = karat.replace('KT','')
            color = color.strip()
            return f'G{karat}{color}'  # e.g., G14Y, G14YW
        else:
            karat = kt_col.replace('KT','')
            return f'G{karat}X'  # unknown color
    return 'UNKNOWN'

# Insert Metal column after OrderQty
df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'Metal',
    df_selected['KT/Col'].apply(metal_code)
)

# Function to extract tone from KT/Col
def extract_tone(kt_col):
    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip()
    if '/' in kt_col:
        return kt_col.split('/')[-1]  # take last part after "/"
    return ''  # if no slash, return blank

# Insert Tone column after Metal
df_selected.insert(
    df_selected.columns.get_loc('Metal') + 1,
    'Tone',
    df_selected['KT/Col'].apply(extract_tone)
)

# Create DesignProductionInstruction by combining CustomerProductionInstruction and Min / Max
def design_instruction(row):
    cpi = '' if pd.isna(row['CustomerProductionInstruction']) else str(row['CustomerProductionInstruction']).strip()
    minmax = '' if pd.isna(row['Min / Max']) else str(row['Min / Max']).strip()
    
    if cpi and minmax:
        return f"{cpi}, {minmax}"
    elif cpi:
        return cpi
    elif minmax:
        return minmax
    else:
        return ''

# Insert DesignProductionInstruction column after Tone
df_selected.insert(
    df_selected.columns.get_loc('Tone') + 1,
    'DesignProductionInstruction',
    df_selected.apply(design_instruction, axis=1)
)

# Function to create SpecialRemarks
def create_special_remarks(row):
    parts = []
    # Add each part if it exists
    if not pd.isna(row['OrderGroup']):
        parts.append(str(row['OrderGroup']).strip())
    if not pd.isna(row['StyleCode']):
        parts.append(str(row['StyleCode']).strip())
    if not pd.isna(row['KT/Col']):
        parts.append(str(row['KT/Col']).strip())
    if not pd.isna(row['LGD QUALITY']):
        parts.append(str(row['LGD QUALITY']).strip())
    if not pd.isna(row['Mined/LGD']):
        parts.append(str(row['Mined/LGD']).strip())
    if not pd.isna(row['Indigo Order#']):
        parts.append(str(row['Indigo Order#']).strip())
    
    return ', '.join(parts)

# Insert SpecialRemarks column after CustomerProductionInstruction
df_selected.insert(
    df_selected.columns.get_loc('CustomerProductionInstruction') + 1,
    'SpecialRemarks',
    df_selected.apply(create_special_remarks, axis=1)
)

df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'OrderItemPcs',
    ''  # blank by default
)

# Move ItemPoNo column after Tone
item_po_no = df_selected.pop('ItemPoNo')
df_selected.insert(df_selected.columns.get_loc('Tone') + 1, 'ItemPoNo', item_po_no)

# List of new columns to add after ItemPoNo
new_cols = ['ItemRefNo', 'StockType', 'MakeType']

# Insert each new column as blank after ItemPoNo
pos = df_selected.columns.get_loc('ItemPoNo') + 1
for col in new_cols:
    df_selected.insert(pos, col, '')
    pos += 1
    
# Move DesignProductionInstruction after SpecialRemarks
design_col = df_selected.pop('DesignProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('SpecialRemarks') + 1, 'DesignProductionInstruction', design_col)

# List of additional new columns to add after SKUNo
additional_cols = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality','Certificate'
]

# Insert each new column as blank after SKUNo
pos = df_selected.columns.get_loc('SKUNo') + 1
for col in additional_cols:
    df_selected.insert(pos, col, '')
    pos += 1

df_selected.drop(columns=['Mined/LGD','Indigo Order#','KT/Col','Min / Max','LGD QUALITY'], inplace=True) 
df_selected.to_csv(r'C:\Users\Admin\Desktop\p_PO_Cleaned.csv', index=False)
df_selected.head()

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,OrderGroup,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,IA4055-PTW-95A,-,10,,PT950,W,Mounting-1,,,...,INDNY-BE,IA4055-PTW-95A,,,,,,,,
1,2,IA4057-PTW-95A,-,7,,PT950,W,Mounting-1,,,...,INDNY-BE,IA4057-PTW-95A,,,,,,,,


In [9]:
# If Earring/ Stud FINAL CODE
import pandas as pd
import re

# --- Step 1: Read Excel ---
df = pd.read_excel(
    r'C:\Users\Admin\Downloads\Po# Mounting-1.xlsx',
    sheet_name='PO Copy.',
    skiprows=1
)

# --- Step 2: Select relevant columns ---
selected_columns = [
    'Vendor Style#', 'SIZE', 'Ord Qty', 'KT/Col', 'NY PO#',
    'Spl. Remark', 'Min / Max', 'Stamp Instr', 'Cust',
    'NY Style #', 'LGD QUALITY', 'Mined/LGD', 'Indigo Order#'
]
df_selected = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df_selected.rename(columns={
    'Vendor Style#': 'StyleCode',
    'SIZE': 'ItemSize',
    'Ord Qty': 'OrderQty',
    'NY PO#': 'ItemPoNo',
    'Spl. Remark': 'CustomerProductionInstruction',
    'Stamp Instr': 'StampInstruction',
    'Cust': 'OrderGroup',
    'NY Style #': 'SKUNo'
}, inplace=True)

# --- Step 4: Clean StyleCode (keep only prefix before first "-") ---
def clean_stylecode(code):
    if pd.isna(code):
        return ''
    code = str(code).strip()
    return code.split('-')[0] if '-' in code else code

df_selected['StyleCode'] = df_selected['StyleCode'].apply(clean_stylecode)

# --- Step 5: Add SrNo ---
df_selected.insert(
    df_selected.columns.get_loc('StyleCode'),
    'SrNo',
    range(1, len(df_selected) + 1)
)

# --- Step 6: Map KT/Col to Metal ---
def metal_code(kt_col):
    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip().replace(' ', '')

    # Platinum
    if '950PT' in kt_col or 'PT950' in kt_col:
        return 'PC95'

    # Karat gold
    if 'KT' in kt_col:
        if '/' in kt_col:
            karat, color = kt_col.split('/')
            karat = karat.replace('KT', '')
            color = color.strip()
            return f'G{karat}{color}'  # e.g., G14Y
        else:
            karat = kt_col.replace('KT', '')
            return f'G{karat}X'
    return 'UNKNOWN'

df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'Metal',
    df_selected['KT/Col'].apply(metal_code)
)

# --- Step 7: Extract Tone ---
def extract_tone(row):
    kt_col = row['KT/Col']
    metal = row['Metal']

    # If Metal is platinum
    if metal == 'PC95':
        return 'PT'

    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip()
    if '/' in kt_col:
        return kt_col.split('/')[-1].strip()
    return ''

df_selected.insert(
    df_selected.columns.get_loc('Metal') + 1,
    'Tone',
    df_selected.apply(extract_tone, axis=1)
)

# --- Step 8: Ask user for ItemSize prefix (e.g., UP / US / leave blank) ---
prefix = input("Enter prefix for ItemSize (e.g., UP or US or leave blank): ").strip().upper()

def format_item_size(size):
    if pd.isna(size) or str(size).strip() == '':
        return ''
    size = str(size).strip()
    return f"{prefix} {size}" if prefix else size

df_selected['ItemSize'] = df_selected['ItemSize'].apply(format_item_size)

# --- Step 9: DesignProductionInstruction ---
def design_instruction(row):
    cpi = '' if pd.isna(row['CustomerProductionInstruction']) else str(row['CustomerProductionInstruction']).strip()
    minmax = '' if pd.isna(row['Min / Max']) else str(row['Min / Max']).strip()
    if cpi and minmax:
        return f"{cpi}, {minmax}"
    elif cpi:
        return cpi
    elif minmax:
        return minmax
    return ''

df_selected.insert(
    df_selected.columns.get_loc('Tone') + 1,
    'DesignProductionInstruction',
    df_selected.apply(design_instruction, axis=1)
)

# --- Step 10: SpecialRemarks ---
def create_special_remarks(row):
    parts = []
    for col in ['OrderGroup', 'StyleCode', 'KT/Col', 'LGD QUALITY', 'Mined/LGD', 'Indigo Order#']:
        val = row.get(col)
        if pd.notna(val) and str(val).strip():
            parts.append(str(val).strip())
    return ', '.join(parts)

df_selected.insert(
    df_selected.columns.get_loc('CustomerProductionInstruction') + 1,
    'SpecialRemarks',
    df_selected.apply(create_special_remarks, axis=1)
)

# --- Step 11: Add OrderItemPcs ---
df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'OrderItemPcs',
    ''
)

# --- Step 12: Move ItemPoNo after Tone ---
item_po_no = df_selected.pop('ItemPoNo')
df_selected.insert(df_selected.columns.get_loc('Tone') + 1, 'ItemPoNo', item_po_no)

# --- Step 13: Add new columns after ItemPoNo ---
new_cols = ['ItemRefNo', 'StockType', 'MakeType']
pos = df_selected.columns.get_loc('ItemPoNo') + 1
for col in new_cols:
    df_selected.insert(pos, col, '')
    pos += 1

# --- Step 14: Move DesignProductionInstruction after SpecialRemarks ---
design_col = df_selected.pop('DesignProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('SpecialRemarks') + 1, 'DesignProductionInstruction', design_col)

# --- Step 15: Add more columns after SKUNo ---
additional_cols = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice',
    'StoneQuality', 'Certificate'
]
pos = df_selected.columns.get_loc('SKUNo') + 1
for col in additional_cols:
    df_selected.insert(pos, col, '')
    pos += 1

# --- Step 16: Drop unwanted columns ---
df_selected.drop(columns=['Mined/LGD', 'Indigo Order#', 'KT/Col', 'Min / Max', 'LGD QUALITY'], inplace=True)

# --- Step 17: Export to Excel ---
output_path = r'C:\Users\Admin\Desktop\PO_MOUNT_Cleaned.xlsx'
df_selected.to_excel(output_path, index=False)
print(f"✅ File exported successfully to: {output_path}")

# --- Step 18: Display top rows ---
df_selected.head()


Enter prefix for ItemSize (e.g., UP or US or leave blank):  


✅ File exported successfully to: C:\Users\Admin\Desktop\PO_MOUNTING_Cleaned.xlsx


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality,Certificate
0,1,IA4055,-,10,,PC95,PT,Mounting-1,,,...,IA4055-PTW-95A,,,,,,,,,
1,2,IA4057,-,7,,PC95,PT,Mounting-1,,,...,IA4057-PTW-95A,,,,,,,,,


In [16]:
#FINAL CODE FOR 23,29
import pandas as pd

# --- Step 1: Read Excel ---
df = pd.read_excel(
    r'C:\Users\Admin\Downloads\JOB WORK INVOICE SHM023 DT19062025 SHIM JOB WORK-LGD.xlsx',
    sheet_name='PO Copy.',
    skiprows=2
)

# --- Step 2: Select relevant columns ---
selected_columns = [
    'Vendor Style#', 'SIZE', 'Ord Qty', 'KT/Col', 'NY PO#',
    'Spl. Remark', 'Min / Max', 'Stamp Instr', 'Cust',
    'NY Style #', 'LGD QUALITY', 'Mined/LGD', 'Indigo Order#'
]
df_selected = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df_selected.rename(columns={
    'Vendor Style#': 'StyleCode',
    'SIZE': 'ItemSize',
    'Ord Qty': 'OrderQty',
    'NY PO#': 'ItemPoNo',
    'Spl. Remark': 'CustomerProductionInstruction',
    'Stamp Instr': 'StampInstruction',
    'Cust': 'OrderGroup',
    'NY Style #': 'SKUNo'
}, inplace=True)

# --- Step 4: Clean StyleCode (keep only before first '-') ---
def clean_stylecode(code):
    if pd.isna(code):
        return ''
    code = str(code).strip()
    return code.split('-')[0] if '-' in code else code

df_selected['StyleCode'] = df_selected['StyleCode'].apply(clean_stylecode)

# --- Step 5: Add SrNo ---
df_selected.insert(
    df_selected.columns.get_loc('StyleCode'),
    'SrNo',
    range(1, len(df_selected) + 1)
)

# --- Step 6: Map KT/Col to Metal code ---
def metal_code(kt_col, tone):
    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip()

    # Platinum
    if '950PT' in kt_col or 'PT950' in kt_col or 'PT 950' in kt_col:
        return 'PC95'

    # Karat gold logic
    if 'KT' in kt_col:
        karat = ''.join([ch for ch in kt_col if ch.isdigit()])
        if tone:
            return f'G{karat}{tone}'
        return f'G{karat}'
    return 'UNKNOWN'

# --- Step 7: Extract Tone ---
def extract_tone(kt_col):
    if pd.isna(kt_col):
        return ''
    kt_col = str(kt_col).upper().strip()
    if '/' in kt_col:
        return kt_col.split('/')[-1].strip()
    if '950PT' in kt_col or 'PT950' in kt_col or 'PT 950' in kt_col:
        return 'PT'
    return ''

df_selected['Tone'] = df_selected['KT/Col'].apply(extract_tone)

# --- Step 8: Apply Metal mapping (using both KT/Col and Tone) ---
df_selected['Metal'] = df_selected.apply(lambda r: metal_code(r['KT/Col'], r['Tone']), axis=1)

# --- Step 9: Clean ItemPoNo (take only before '-') ---
df_selected['ItemPoNo'] = df_selected['ItemPoNo'].astype(str).apply(lambda x: x.split('-')[0] if '-' in x else x)

# --- Step 10: Format ItemSize with prefix and zero-padding ---
prefix = input("Enter ItemSize prefix (e.g., UP, US, or leave blank): ").strip().upper()

def format_item_size(size):
    if pd.isna(size) or str(size).strip() == '':
        return ''
    size_str = str(size).strip()
    try:
        num = float(size_str)
        if num.is_integer():  # e.g., 7.0
            num_str = f"{int(num):02d}"  # pad single digits with 0
        else:
            num_str = str(num).rstrip('0').rstrip('.')  # remove .0
    except ValueError:
        num_str = size_str
    return f"{prefix}{num_str}" if prefix else num_str

df_selected['ItemSize'] = df_selected['ItemSize'].apply(format_item_size)

# --- Step 11: DesignProductionInstruction ---
def design_instruction(row):
    cpi = '' if pd.isna(row['CustomerProductionInstruction']) else str(row['CustomerProductionInstruction']).strip()
    minmax = '' if pd.isna(row['Min / Max']) else str(row['Min / Max']).strip()
    if cpi and minmax:
        return f"{cpi}, {minmax}"
    elif cpi:
        return cpi
    elif minmax:
        return minmax
    return ''

df_selected.insert(
    df_selected.columns.get_loc('Tone') + 1,
    'DesignProductionInstruction',
    df_selected.apply(design_instruction, axis=1)
)

# --- Step 12: SpecialRemarks ---
def create_special_remarks(row):
    parts = []
    for col in ['OrderGroup', 'StyleCode', 'KT/Col', 'LGD QUALITY', 'Mined/LGD', 'Indigo Order#']:
        val = row.get(col)
        if pd.notna(val) and str(val).strip():
            parts.append(str(val).strip())
    return ', '.join(parts)

df_selected.insert(
    df_selected.columns.get_loc('CustomerProductionInstruction') + 1,
    'SpecialRemarks',
    df_selected.apply(create_special_remarks, axis=1)
)

# --- Step 13: Add OrderItemPcs ---
df_selected.insert(
    df_selected.columns.get_loc('OrderQty') + 1,
    'OrderItemPcs', value=1
)

# --- Step 14: Move ItemPoNo after Tone ---
item_po_no = df_selected.pop('ItemPoNo')
df_selected.insert(df_selected.columns.get_loc('Tone') + 1, 'ItemPoNo', item_po_no)

# --- Step 15: Add new columns after ItemPoNo ---
new_cols = ['ItemRefNo', 'StockType', 'MakeType']
pos = df_selected.columns.get_loc('ItemPoNo') + 1
for col in new_cols:
    df_selected.insert(pos, col, '')
    pos += 1

# --- Step 16: Move DesignProductionInstruction after SpecialRemarks ---
design_col = df_selected.pop('DesignProductionInstruction')
df_selected.insert(df_selected.columns.get_loc('SpecialRemarks') + 1, 'DesignProductionInstruction', design_col)

# --- Step 17: Add additional columns after SKUNo ---
additional_cols = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality', 'Certificate'
]
pos = df_selected.columns.get_loc('SKUNo') + 1
for col in additional_cols:
    df_selected.insert(pos, col, '')
    pos += 1

# --- Step 18: Drop unwanted columns ---
df_selected.drop(columns=['Mined/LGD', 'Indigo Order#', 'KT/Col', 'Min / Max', 'LGD QUALITY'], inplace=True)

# --- Step 19: Export to Excel ---
output_path = r'C:\Users\Admin\Desktop\SHM023_DT19062025_PO_Cleaned.xlsx'
df_selected.to_excel(output_path, index=False)
print(f"✅ Excel file saved to: {output_path}")

# --- Step 20: Display preview ---
df_selected.head()


Enter ItemSize prefix (e.g., UP, US, or leave blank):  UP


✅ Excel file saved to: C:\Users\Admin\Desktop\SHM023_DT19062025_PO_Cleaned.xlsx


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,CustomerProductionInstruction,SpecialRemarks,DesignProductionInstruction,StampInstruction,OrderGroup,...,Expecteddeliverydate,SetPrice,StoneQuality,Certificate,Tone,ItemPoNo,ItemRefNo,StockType,MakeType,Metal
0,1,VR4178SEB1,UP07,7,1,DIP RHODIUM,"REED, VR4178SEB1, 14KT/W, PTP I/SI2 (GSI),RD I...","DIP RHODIUM, REQ GOLD WT 3.18 GMS (MIN/MAX)(3....",KT + IDI LOGO + LGD,REED,...,,,,,W,208538,,,,G14W
1,2,XR3843L,UP07,1,1,DIP RHODUIM/ NO JEWELLERY CERTIFICATION REQUIR...,"HZ, XR3843L, 10KT / W, OVL H/VS2 (LGD),RD F/VS...",DIP RHODUIM/ NO JEWELLERY CERTIFICATION REQUIR...,KT+IDI LOGO+LGD+HDS,HZ,...,,,,,W,408815,,,,G10W
2,3,XR5357ME,UP10.5,1,1,NO RHODIUM -F/SI2] - IGI - Evaluation Report +...,"RH, XR5357ME, 14KT / Y, EMR F/SI2 (LGD),RD F/S...",NO RHODIUM -F/SI2] - IGI - Evaluation Report +...,KT + IDI LOGO + LGD,RH,...,,,,,Y,408817,,,,G14Y
3,4,WR4335JG,UP10,1,1,YELLOW GOLD & WHITE GOLD AS PER IMAGE/Spl Finish,"HZ, WR4335JG, 14KT / YW, PT F/VS2 (LGD), LGD, ...",YELLOW GOLD & WHITE GOLD AS PER IMAGE/Spl Fini...,KT + IDI LOGO + LCD,HZ,...,,,,,YW,409510,,,,G14YW
4,5,SVBRC24367XX,UP6.75,1,1,NO RHODIUM / CHAIN LENGH 2.0 INCH,"IDI, SVBRC24367XX, PT 950 / W, PTW 13CTTW F/VS...","NO RHODIUM / CHAIN LENGH 2.0 INCH, REQ GOLD W...",PLAT + IDI LOGO + Lab Created,IDI,...,,,,,W,208704,,,,PC95
